In [9]:
import pandas as pd
import random
import numpy as np
import k3d
def gen_surface(xymin,xymax,step):
    x=np.arange(xymin,xymax,step)
    y=np.arange(xymin,xymax,step)
    X,Y = np.meshgrid(x,y)
    xx=X.flatten()
    yy = Y.flatten()
    zz = yy*0

    points = np.zeros((xx.shape[0],3))
    points[:,0]=xx
    points[:,1]=yy
    points[:,2]=zz
    return points


def patch(dataset,randomP,num):
    pcl = np.empty((num*len(randomP),3))
    # pcl=dataset
    r =1
    start =0
    for i in randomP:
        num1=int(num*random.choice(np.linspace(0.1,1.30)))
        p=dataset[int(i),0:3]
        d=np.linalg.norm((dataset[:,:3]-p),axis=1)
        sort_id = np.argsort(d)
        rr=np.linspace(0.002,0,num1)
        sq=dataset[sort_id[:num1],:]
        sq[:,2]=sq[:,2]-rr
        dataset[sort_id[:num1]]=sq

        pcl[start:start+num1,:]=np.asarray(sq[:num1,:])
        start = start +num1
    return pcl,dataset

def farthest_point_sample(point, npoint):
    """
    Input:
        xyz: pointcloud data, [N, D]
        npoint: number of samples
    Return:
        centroids: sampled pointcloud index, [npoint, D]
    """
    N, D = point.shape
    xyz = point[:,:3]
    centroids = np.zeros((npoint,))
    distance = np.ones((N,)) * 1e10
    farthest = np.random.randint(0, N)
    for i in range(npoint):
        centroids[i] = farthest
        centroid = xyz[farthest, :]
        dist = np.sum((xyz - centroid) ** 2, -1)
        mask = dist < distance
        distance[mask] = dist[mask]
        farthest = np.argmax(distance, -1)
    point = point[centroids.astype(np.int32)]
    return point,centroids


point = gen_surface(0,0.3,0.001)
a,b=farthest_point_sample(point, 10)
pcl,dataset = patch(point,b,200)
plot = k3d.plot()

plot +=k3d.points(dataset[:,:3], point_size=0.0005)#, attribute=dataset[:,2], color_map=k3d.colormaps.basic_color_maps.Jet)
#, attribute=scan[:,3], color_map=k3d.colormaps.basic_color_maps.Jet)
plot.display()

/home/kumaken/.local/lib/python3.8/site-packages/traittypes/traittypes.py:97: UserWarning: Given trait value dtype "float64" does not match required type "float32". A coerced copy has been created.
  warnings.warn(


Output()

In [6]:
from scipy.spatial import Delaunay
import open3d as o3d
from functools import reduce
def get_triangles_vertices(triangles, vertices):
    triangles_vertices = []
    for triangle in triangles:
        new_triangles_vertices = [vertices[triangle[0]], vertices[triangle[1]], vertices[triangle[2]]]
        triangles_vertices.append(new_triangles_vertices)
    return np.array(triangles_vertices)

def volume_under_triangle(triangle):
    p1, p2, p3 = triangle
    x1, y1, z1 = p1
    x2, y2, z2 = p2
    x3, y3, z3 = p3
    return abs((z1+z2+z3)*(x1*y2-x2*y1+x2*y3-x3*y2+x3*y1-x1*y3)/6)



In [10]:
xy_catalog = []
xyz =dataset
for point in xyz:
    xy_catalog.append([point[0], point[1]])
tri = Delaunay(np.array(xy_catalog))
surface = o3d.geometry.TriangleMesh()
surface.vertices = o3d.utility.Vector3dVector(xyz)
surface.triangles = o3d.utility.Vector3iVector(tri.simplices)
# o3d.visualization.draw_geometries([surface], mesh_show_wireframe=True)
volume = reduce(lambda a, b:  a + volume_under_triangle(b), get_triangles_vertices(surface.triangles, surface.vertices), 0)
print(f"volumes: {volume} m3")
import pyransac3d as pyrsc
plane = pyrsc.Plane()
best_eq, best_inliers= plane.fit(dataset, thresh=0.0005)
iid = np.arange(len(dataset))
d = np.delete(iid,best_inliers)
cc=dataset[d,:]
plot = k3d.plot()
# plot +=k3d.points(point[:,:3], point_size=0.001)
plot +=k3d.points(dataset[best_inliers,:3], point_size=0.001)#, attribute=dataset[best_inliers,2], color_map=k3d.colormaps.basic_color_maps.Jet)
plot +=k3d.points(dataset[d,:3], point_size=0.001,  color = 0xff00ff)
plot.display()

import hdbscan
clustering = hdbscan.HDBSCAN(min_samples=5).fit(dataset[d,:3])
result = clustering.labels_
n_clusters = len(set(result))
print("clack num:",n_clusters)


volumes: 9.959115065859845e-07 m3


Output()

clack num: 10
